In [ ]:
import gzip

### 读取待抽取Freebase图谱实体ID

In [21]:
FreebaseID = set()
with open("douban2fb.txt", "r", encoding="utf-8") as file:
    for line in file:
        FreebaseID.add(line.split("\t")[1].strip())

### 初始化pd结构存储三元结构

In [ ]:
import pandas as pd
pd_triplet = pd.DataFrame()
pd_triplet['head'] = None
pd_triplet['relation'] = None
pd_triplet['tail'] = None
pd_triplet.to_csv("freebase_douban.csv", index=False)

### 匹配实体

In [20]:
ori_FreebaseID = FreebaseID.copy()

In [ ]:
def match_entity(ID)-> bool :
    if ID in ori_FreebaseID:
        return True
    else:
        return False

### 找到与豆瓣ID映射实体相同的triplet(一跳)

In [ ]:
j = 0
with gzip.open("freebase_douban.gz") as f:
    for line in f:
        line = line.strip()
        triplet = line.decode().split('\t')[:3]
        if match_entity(triplet[0].strip("<>").split('/')[-1]):
            new_row = {"head":triplet[0].strip("<>"),"relation":triplet[1].strip("<>"),"tail":triplet[2].strip("<>")}
            pd_triplet = pd_triplet.append(new_row,ignore_index=True)
            if(triplet[2].startswith("<http://rdf.freebase.com/ns/")):
                FreebaseID.add(triplet[2].strip("<>").split('/')[-1])
        j += 1
        if(j % 1000000==0):
          pd_triplet.to_csv('freebase_douban.csv', mode='a', index=False, header = False)
          temp = pd.read_csv('freebase_douban.csv')
          print(j)
          print("Length of pd_triplet:", len(temp))
          pd_triplet = pd_triplet[0:0]
          
pd_triplet.to_csv('freebase_douban.csv', mode='a', index=False, header = False)
temp = pd.read_csv('freebase_douban.csv')
print(j)
print("Length of pd_triplet:", len(temp))

In [ ]:
with open("first_entity", 'w') as file:
    for element in FreebaseID:
        file.write(str(element) + '\n')
FreebaseID.clear()

In [ ]:
FreebaseID.clear()
pd_triplet = pd.read_csv('first_freebase_douban.csv')
with open("first_entity", 'r') as file:
    FreebaseID = {line.strip() for line in file}

### 筛选实体,得到第二跳的实体

In [ ]:
#统计不同实体出现次数
entity_num = {key: 0 for key in FreebaseID}
relation_num = dict()
for index, row in pd_triplet.iterrows():
    head = row['head']
    relation = row['relation']
    tail = row['tail']
    if head.split('/')[-1] in entity_num:
        entity_num[head.split('/')[-1]] += 1
    if tail.split('/')[-1] in entity_num:
        entity_num[tail.split('/')[-1]] += 1
    if relation in relation_num:
        relation_num[relation] += 1
    else:
        relation_num[relation] = 0
to_be_delete = []
# 保留了至少出现在 20 个三元组中的实体，同时只保留出现超过 50 次的关系
for index, row in pd_triplet.iterrows():
    if relation_num[row['relation']] < 50 :
        to_be_delete.append(index)
pd_triplet = pd_triplet.drop(to_be_delete)
# 得到第二跳的实体
FreebaseID.clear()
for index, row in pd_triplet.iterrows():
    if row['head'].startswith("http://rdf.freebase.com/ns/") and entity_num[row['head'].split('/')[-1]] >= 20:
        FreebaseID.add(row['head'].split('/')[-1])
    if row['tail'].startswith("http://rdf.freebase.com/ns/") and entity_num[row['tail'].split('/')[-1]] >= 20:
        FreebaseID.add(row['tail'].split('/')[-1])

In [ ]:
with open("first_selected_entity", 'w') as file:
    for element in FreebaseID:
        file.write(str(element) + '\n')
FreebaseID.clear()

### 找到与豆瓣ID映射实体相同的triplet(二跳)

In [4]:
import pandas as pd
pd_triplet = pd.DataFrame()
pd_triplet['head'] = None
pd_triplet['relation'] = None
pd_triplet['tail'] = None

In [5]:
import pandas as pd
pd_triplet = pd.DataFrame()
pd_triplet['head'] = None
pd_triplet['relation'] = None
pd_triplet['tail'] = None
pd_triplet.to_csv("second_freebase_douban.csv", index=False)

In [6]:
with open("first_selected_entity", 'r') as file:
    second_ori_FreebaseID = {line.strip() for line in file}
second_FreebaseID = second_ori_FreebaseID.copy()

In [ ]:
import gzip
j = 0
entity_num = {key: 0 for key in second_FreebaseID}
with gzip.open("freebase_douban.gz") as f:
    for line in f:
        line = line.strip()
        triplet = line.decode().split('\t')[:3]
        if triplet[0].strip("<>").split('/')[-1]  in second_ori_FreebaseID:
            new_row = {"head":triplet[0].strip("<>"),"relation":triplet[1].strip("<>"),"tail":triplet[2].strip("<>")}
            pd_triplet = pd_triplet._append(new_row,ignore_index=True)
            if(triplet[2].startswith("<http://rdf.freebase.com/ns/")):
                second_FreebaseID.add(triplet[2].strip("<>").split('/')[-1])
            entity_num[triplet[0].strip("<>").split('/')[-1]] += 1
            if entity_num[triplet[0].strip("<>").split('/')[-1]] >= 20000:  #有的实体出现太多次了！
              second_ori_FreebaseID.remove(triplet[0].strip("<>").split('/')[-1])
              print("i have removed:",triplet[0].strip("<>").split('/')[-1])
        j += 1
        if(j % 1000000==0):
          temp = pd.read_csv('second_freebase_douban.csv')
          print(j)
          print("Length of pd_triplet:", len(temp))
        if( len(pd_triplet) > 10000):
          pd_triplet.to_csv('second_freebase_douban.csv', mode='a', index=False, header = False)
          pd_triplet = pd_triplet[0:0]
          
pd_triplet.to_csv('second_freebase_douban.csv', mode='a', index=False, header = False)
temp = pd.read_csv('second_freebase_douban.csv')
print(j)
print("Length of pd_triplet:", len(temp))

In [28]:
with open("second_entity", 'w') as file:
    for element in second_FreebaseID:
        file.write(str(element) + '\n')
second_FreebaseID.clear()

### 筛选第二跳实体

In [22]:
import pandas as pd
pd_triplet = pd.read_csv('first_freebase_douban.csv')
with open("second_entity", 'r') as file:
    second_FreebaseID = {line.strip() for line in file}

In [23]:
pd_triplet = pd.read_csv('second_freebase_douban.csv')
#统计不同实体出现次数
second_entity_num = {key: 0 for key in second_FreebaseID}
second_relation_num = dict()
for index, row in pd_triplet.iterrows():
    head = row['head']
    relation = row['relation']
    tail = row['tail']
    if head.split('/')[-1] in second_entity_num:
        second_entity_num[head.split('/')[-1]] += 1
    if tail.split('/')[-1] in second_entity_num:
        second_entity_num[tail.split('/')[-1]] += 1
    if relation in second_relation_num:
        second_relation_num[relation] += 1
    else:
        second_relation_num[relation] = 0
to_be_delete = []
# 保留了至少出现在 20 个三元组中的实体，同时只保留出现超过 50 次的关系
for index, row in pd_triplet.iterrows():
    flag = (row['head'].split('/')[-1] in second_entity_num and 19500 >= second_entity_num[row['head'].split('/')[-1]] >= 18) and (row['tail'].split('/')[-1] in second_entity_num and 19500 >= second_entity_num[row['tail'].split('/')[-1]] >= 18)
    if second_relation_num[row['relation']] < 50 or not flag :
        to_be_delete.append(index)
pd_triplet = pd_triplet.drop(to_be_delete)
# 得到第二跳的实体
second_FreebaseID.clear()
for index, row in pd_triplet.iterrows():
    if row['head'].startswith("http://rdf.freebase.com/ns/") :
        second_FreebaseID.add(row['head'].split('/')[-1])
    if row['tail'].startswith("http://rdf.freebase.com/ns/") :
        second_FreebaseID.add(row['tail'].split('/')[-1])

# 得到第二跳剩余的关系：
second_relation_num.clear()
for index, row in pd_triplet.iterrows():
    head = row['head']
    relation = row['relation']
    tail = row['tail']
    if relation in second_relation_num:
        second_relation_num[relation] += 1
    else:
        second_relation_num[relation] = 0

In [24]:
for element in FreebaseID:
    if element not in second_FreebaseID:
        print("no")

In [25]:
len(second_FreebaseID)

1898

In [26]:
len(pd_triplet)

42701

In [18]:
with open("movie_relation", 'w') as file:
    for key, value in second_relation_num.items():
        # if value >= 50:
        file.write(f"{key}: {value}\n")

In [19]:
with open("second_selected_entity", 'w') as file:
    for element in second_FreebaseID:
        file.write(str(element) + '\n')
second_FreebaseID.clear()
pd_triplet.to_csv("second_selected_freebase_douban.csv",index=False)